In [1]:
import aux.sao_paulo_shapefile as sp_shp
import aux.sao_paulo_graph as sp_graph
import aux.preprocess_congestion as cong
import aux.preprocess_accidents as acc
import folium
from scipy.spatial import KDTree

# se nenessário recarregar os arquivos auxiliares
from importlib import reload
reload(sp_shp)
reload(sp_graph)
reload(cong)
reload(acc)
import numpy as np
import pandas as pd
from datetime import datetime

#### Criação do Grafo (5 min)

In [2]:
print('{}\t Fazendo leitura do arquivo (1 min)'.format(datetime.now().strftime('%H:%M:%S')))
gdf = sp_shp.read_and_process_file('data/logradouros/SIRGAS_SHP_logradouronbl_line.shp')

print('{}\t Iniciando criação do grafo (1 min)'.format(datetime.now().strftime('%H:%M:%S')))
G = sp_graph.create_and_process_graph (gdf)

print('{}\t Fim da criação do grafo'.format(datetime.now().strftime('%H:%M:%S')))

22:03:00	 Fazendo leitura do arquivo (1 min)
22:03:21	 Iniciando criação do grafo (1 min)
22:04:22	 Fim da criação do grafo


#### Inserção dos dados de Acidentes

In [3]:
print('{}\tIniciando atribuição do acidentes nas arestas (1 min)'.format(datetime.now().strftime('%H:%M:%S')))

df_acidentes = pd.read_csv('data/acidentes/df_acidentes.csv')
df_acidentes = df_acidentes[['chave', 'data', 'hora', 'dia_semana',
                'latitude', 'longitude', 'tipo_acidente', 'qtde_vitimas']]
df_acidentes['index'] = range(len(df_acidentes))
print('{}\tBase de acidentes: {} registros'.format(datetime.now().strftime('%H:%M:%S'), (len(df_acidentes)))) # 216010 
      
acc.put_accidents_on_edges(G, df_acidentes)
print('{}\tFinalizando atribuição do acidentes nas arestas'.format(datetime.now().strftime('%H:%M:%S')))

22:04:22	Iniciando atribuição do acidentes nas arestas (1 min)
22:04:24	Base de acidentes: 216010 registros
22:05:11	Finalizando atribuição do acidentes nas arestas


#### Inserção dos dados de Congestionamentos

In [4]:
df_congestionamentos = pd.read_csv('data/congestionamentos_pre_processado.csv')
df_congestionamentos.fillna('', inplace = True)
df_congestionamentos['index'] = range(len(df_congestionamentos))
print('{}\tBase de acidentes: {} registros'.format(datetime.now().strftime('%H:%M:%S'), (len(df_congestionamentos)))) #832525 

df_pontos = pd.read_csv('data/coordenadas_congestionamentos.csv')
df_pontos = df_pontos.fillna('')
print('{}\tBase de pontos de referencia: {} registros'.format(datetime.now().strftime('%H:%M:%S'), (len(df_pontos)))) #3447  

C:\Users\gigec\AppData\Local\Temp\ipykernel_3984\569977181.py:1: DtypeWarning: Columns (5,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df_congestionamentos = pd.read_csv('data/congestionamentos_pre_processado.csv')


22:05:18	Base de acidentes: 832525 registros
22:05:18	Base de pontos de referencia: 3447 registros


In [5]:
print('{}\t Identificando as arestas de cada corredor (5 min) '.format(datetime.now().strftime('%H:%M:%S')))
arestas_por_corredor = cong.get_dict_edges_corridors (G, df_congestionamentos)

def get_subgraphs (G, edges_dict, neighborhood_depth = 6):
    subgraphs = { }
    for corridor, e in edges_dict.items():
        subgraphs[corridor] = sp_graph.generate_subgraph(G, e['arestas'], neighborhood_depth)
    return subgraphs

print('{}\t Particionando subgrafos (1 min)'.format(datetime.now().strftime('%H:%M:%S')))

subgraphs = get_subgraphs(G, arestas_por_corredor)

print('{}\t Fim do processamento dos corredores'.format(datetime.now().strftime('%H:%M:%S')))

22:05:18	 Identificando as arestas de cada corredor (5 min) 
22:08:36	 Particionando subgrafos (1 min)
22:08:49	 Fim do processamento dos corredores


### Geração da base de arestas

In [6]:
def gera_arestas_acidentes(arestas_do_corredor, acidentes_inseridos):
    edges_data = []
    n = len(arestas_do_corredor) 
    print(f"{datetime.now().strftime('%H:%M:%S')}\tInicio acidentes \t\t{n} arestas")
    
    i = 0
    for e in arestas_do_corredor:
        if 'acidentes' in e[2]:
            for a in e[2]['acidentes']:
                if a[0] not in acidentes_inseridos:
                    i += 1
                    acidentes_inseridos.add(a[0])
                    edges_data.append([e[0], e[1], e[2]['geometry'], *a, None])
                    
    print(f"{datetime.now().strftime('%H:%M:%S')}\tFim acidentes \t\t\t{i} acidentes inseridos")
    return edges_data

In [14]:
def encontra_no_mais_proximo(ponto, nodes):
    # necessario por conta de erros numericos D:
    tree = KDTree(nodes)
    d, i = tree.query(ponto)
    return nodes[i]

def obtem_ponto(subgraph, df_pontos, corredor, referencia, tipo, distancia):
    ponto = eval(df_pontos.loc[
        df_pontos['corredor']  == corredor].loc[
        df_pontos['ponto']     == referencia].loc[
        df_pontos['tipo']      == tipo].loc[
        df_pontos['distancia'] == distancia].iloc[0]['vertice'])
    
    ponto = (round(ponto[0], 10), round(ponto[1], 10))
    
    if ponto not in subgraph.nodes():
        ponto = encontra_no_mais_proximo(ponto, list(subgraph.nodes()))
    
    return ponto

def gera_arestas_caminho(subgraph, df_pontos, caminho, corredor, erros):
    ponto_inicial = obtem_ponto(subgraph, df_pontos, corredor, caminho[0], caminho[1], caminho[2])
    ponto_final = obtem_ponto(subgraph, df_pontos, corredor, caminho[3], caminho[4], caminho[5])

    try:
        S = cong.encontra_caminho(subgraph, ponto_inicial, ponto_final)
    except Exception as e:
        erros.append([corredor, *caminho, e])
        print('--- erro ---', len(erro))
        return []
    return S.edges(data = True)
    
    
def gera_arestas_congestionamentos(df_pontos, df_congestionamentos, subgraph, corredor, erros):
    edges_data = []
    df_cong = df_congestionamentos.query(f'corredor == "{corredor}"')
    
    df_pontos_aux = df_pontos.loc[df_pontos['corredor'] == corredor]
    
    caminhos = df_cong.groupby(['referencia_a', 'tipo_a', 'distancia_a',
                                'referencia_z', 'tipo_z', 'distancia_z'])
    
    print(f"{datetime.now().strftime('%H:%M:%S')}\tInicio congestionamentos \t{len(caminhos)} caminhos")
    
    arestas_congestionamentos = []

    for caminho, congs in caminhos:
        arestas_caminho = gera_arestas_caminho(subgraph, df_pontos, caminho, corredor, erros)
        arestas_congestionamentos.extend(arestas_caminho)
        for e in arestas_caminho:
            for _, c in congs.iterrows():
                [data, hora] = c['data'].split(' ') if ' ' in c['data'] else [c['data'], None]
                edges_data.append([e[0], e[1], e[2]['geometry'], 'c_' + str(c['index']),
                                   data, hora, None, 
                                   None, None,
                                   c['tamanho']])
                
    subgafo_restrito = sp_graph.generate_subgraph(subgraph, arestas_congestionamentos, 4)

    print(f"{datetime.now().strftime('%H:%M:%S')}\tFim congestionamentos \t\t{len(df_cong)} congestionamentos")
    return edges_data, subgafo_restrito

In [15]:
# tempo estimado de processamento: 30 minutos

acidentes_inseridos = set()
erros = []
total = len(subgraphs)
i = 0
for S in subgraphs:
    print('*'*100)
    c = S.split('/')[0].strip()
    print(f'{i}/{total}\t{c}')
    i += 1
    
    arestas_congestionamentos, subgrafo_restrito = gera_arestas_congestionamentos(df_pontos, df_congestionamentos, 
                                                               subgraphs[S], S.strip(), erros)

    arestas_acidentes = gera_arestas_acidentes(subgrafo_restrito.edges(data = True), acidentes_inseridos)
    
    arestas = arestas_congestionamentos + arestas_acidentes
    
    df_arestas = pd.DataFrame(data = arestas, 
                              columns = ['vertice_a', 'vertice_b', 'geometry', 'index', 
                                         'data', 'hora', 'dia_semana', 
                                         'tipo_acidente', 'qtde_vitimas',
                                         'tamanho_congestionamento'])
    
    df_arestas.to_csv('data/arestas/arestas_' + c + '.csv')
    print(f"{datetime.now().strftime('%H:%M:%S')}\tFim do processamento do corredor")
    

****************************************************************************************************
0/167	AL STO AMARO
22:11:49	Inicio congestionamentos 	1 caminhos
22:11:49	Fim congestionamentos 		9 congestionamentos
22:11:49	Inicio acidentes 		172 arestas
22:11:49	Fim acidentes 			764 acidentes inseridos
22:11:50	Fim do processamento do corredor
****************************************************************************************************
1/167	AV  I INTERLAGOS
22:11:50	Inicio congestionamentos 	164 caminhos
22:12:00	Fim congestionamentos 		5591 congestionamentos
22:12:00	Inicio acidentes 		1367 arestas
22:12:00	Fim acidentes 			3717 acidentes inseridos
22:12:08	Fim do processamento do corredor
****************************************************************************************************
2/167	AV ADOLFO PINHEIRO
22:12:08	Inicio congestionamentos 	4 caminhos
22:12:08	Fim congestionamentos 		24 congestionamentos
22:12:08	Inicio acidentes 		202 arestas
22:12:08	Fim acidente

#### Conferindo quais acidentes ficaram dentro e fora do perímetro dos acidentes

In [16]:
m = folium.Map(
        zoom_start=12,location=(-23.6, -46.6), tiles="cartodb positron")    

n = len(df_acidentes)
qtde_inseridos = 0
i = 0
last_checkpoint = 0

for _, a in df_acidentes.iterrows():

    i += 1
    if i / n > last_checkpoint:
        print('{}\t{}% \t\t({}/{})'.format(datetime.now().strftime('%H:%M:%S'),
                                            int(last_checkpoint * 100), i, n))
        last_checkpoint += .33
            
    #plota acidente
    v_a = (float(a['latitude'].replace(',','.')), float(a['longitude'].replace(',','.')))
    
    if a['index'] in acidentes_inseridos:
        qtde_inseridos += 1
        c = 'blue'
    else: c = 'red'


    folium.Circle(
        location = v_a,
        radius = 1,
        color = c,
        weight = 1
    ).add_to(m)
    
print('{}\t\tSalvando mapa em html (5 min)'.format(datetime.now().strftime('%H:%M:%S')))
m.save('mapas/acidentes_congestionamentos.html')

print('{}\t\tFim do Processamento'.format(datetime.now().strftime('%H:%M:%S')))

print('')
print('Total de Acidentes de São Paulo: ', n)
print('Acidentes Considerados no Estudo: ', qtde_inseridos)

22:42:25	0% 		(1/216010)
22:42:33	33% 		(71284/216010)
22:42:42	66% 		(142567/216010)
22:42:51	99% 		(213850/216010)
22:42:51		Salvando mapa em html (5 min)
22:45:07		Fim do Processamento

Total de Acidentes de São Paulo:  216010
Acidentes Considerados no Estudo:  82774


#### Unindo os csvs

In [17]:
print('{}\tIniciando leitura das bases de cada corredor (1 min)'.format(datetime.now().strftime('%H:%M:%S')))

dfs = []

for S in subgraphs:
    c = S.split('/')[0]
    
    aux = pd.read_csv('data/arestas/arestas_' + c + '.csv')
    dfs.append(aux)

# Concatenar todos os dataframes em um único dataframe
df_arestas = pd.concat(dfs, ignore_index=True)

print('{}\tFim da leitura das bases: {} registros'.format(datetime.now().strftime('%H:%M:%S'), len(df_arestas)))
# 5363089 
# 5349918 

22:51:25	Iniciando leitura das bases de cada corredor (1 min)


C:\Users\gigec\AppData\Local\Temp\ipykernel_3984\3918083833.py:8: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  aux = pd.read_csv('data/arestas/arestas_' + c + '.csv')
C:\Users\gigec\AppData\Local\Temp\ipykernel_3984\3918083833.py:8: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  aux = pd.read_csv('data/arestas/arestas_' + c + '.csv')
C:\Users\gigec\AppData\Local\Temp\ipykernel_3984\3918083833.py:8: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  aux = pd.read_csv('data/arestas/arestas_' + c + '.csv')
C:\Users\gigec\AppData\Local\Temp\ipykernel_3984\3918083833.py:8: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  aux = pd.read_csv('data/arestas/arestas_' + c + '.csv')
C:\Users\gigec\AppData\Local\Temp\ipykernel_3984\3918083833.py:8: DtypeWarning: Columns (7,8) ha

22:51:42	Fim da leitura das bases: 5349918 registros


C:\Users\gigec\AppData\Local\Temp\ipykernel_3984\3918083833.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_arestas = pd.concat(dfs, ignore_index=True)


#### Regulação do campo de data e hora

In [18]:
n = len(df_arestas)

def separa_hora(h):
    if type(h) != str:
        return None
    s = h.split(':')
    if len(s) >= 2:
        return s[0].zfill(2)
    return None

def separa_minuto(h):
    if type(h) != str:
        return None
    s = h.split(':')
    if len(s) >= 2:
        return s[1].zfill(2)
    return None

print(f'{datetime.now().strftime("%H:%M:%S")}\tIniciando conversão dos campos de horario (1 min)')

df_arestas['hora_valor'] = df_arestas['hora'].apply(separa_hora)
df_arestas['minuto_valor'] = df_arestas['hora'].apply(separa_minuto)

df_arestas.dropna(subset=['hora_valor'], inplace = True)
df_arestas.dropna(subset=['minuto_valor'], inplace = True)

print(f'{datetime.now().strftime("%H:%M:%S")}\tFim processamento')
print(f'{n - len(df_arestas)} registros deletados (sem dados de horario)') # 157

22:51:42	Iniciando conversão dos campos de horario (1 min)
22:51:52	Fim processamento
157 registros deletados (sem dados de horario)


In [19]:
n = len(df_arestas)

def separa_dia(d):
    if type(d) != str:
        return None
    sep = '/' if '/' in d else '-'
    s = d.split(sep)
    if len(s) < 2:
        return None
    
    if len(s[0]) == 4:
        return s[2].zfill(2)
    
    return s[0].zfill(2)

def separa_mes(d):
    if type(d) != str:
        return None
    sep = '/' if '/' in d else '-'
    s = d.split(sep)
    if len(s) < 2:
        return None
    return s[1].zfill(2)

def separa_ano(d):
    if type(d) != str:
        return None
    sep = '/' if '/' in d else '-'
    s = d.split(sep)
    if len(s) < 2:
        return None
    
    if len(s[0]) == 4:
        return s[0]
    
    if len(s[2]) == 2:
        return '20' + s[2]
    if len(s[2]) == 4:
        return s[2]
    return None

print(f'{datetime.now().strftime("%H:%M:%S")}\tIniciando conversão dos campos de data (1 min)')

df_arestas['dia_valor'] = df_arestas['data'].apply(separa_dia)
df_arestas['mes_valor'] = df_arestas['data'].apply(separa_mes)
df_arestas['ano_valor'] = df_arestas['data'].apply(separa_ano)

df_arestas.dropna(subset=['dia_valor'], inplace = True)
df_arestas.dropna(subset=['mes_valor'], inplace = True)
df_arestas.dropna(subset=['ano_valor'], inplace = True)

print(f'{datetime.now().strftime("%H:%M:%S")}\tFim processamento')
print(f'{n - len(df_arestas)} registros deletados (com data inconsistente)') # 0

22:51:52	Iniciando conversão dos campos de data (1 min)
22:52:15	Fim processamento
0 registros deletados (com data inconsistente)


In [20]:
df_arestas.sample(5)

,Unnamed: 0,vertice_a,vertice_b,geometry,index,data,hora,dia_semana,tipo_acidente,qtde_vitimas,tamanho_congestionamento,media_grau,hora_valor,minuto_valor,dia_valor,mes_valor,ano_valor
3336118,105617,"(-46.5805595999, -23.5412749664)","(-46.5804399899, -23.5416930341)",LINESTRING (-46.580559599940386 -23.5412749663...,c_332423,13/5/19,19:30,NaN,NaN,NaN,1190.0,NaN,19,30,13,05,2019
1890585,123561,"(-46.6799119503, -23.6543632335)","(-46.6799457119, -23.653977671)",LINESTRING (-46.679945711926116 -23.6539776709...,c_161700,07/12/2021,09:30,NaN,NaN,NaN,1300.0,NaN,09,30,07,12,2021
669385,286164,"(-46.6268520693, -23.5426967087)","(-46.6268501114, -23.5418705708)",LINESTRING (-46.626850111421895 -23.5418705708...,c_88105,05/12/2022,11:00,NaN,NaN,NaN,1680.0,NaN,11,00,05,12,2022
3699239,57530,"(-46.6627815239, -23.6230539469)","(-46.6638688985, -23.6231362424)",LINESTRING (-46.663868898549715 -23.6231362423...,c_421171,27/10/2023,16:00,NaN,NaN,NaN,710.0,NaN,16,00,27,10,2023
44687,43770,"(-46.7009710875, -23.6961568606)","(-46.7000314232, -23.6958558424)",LINESTRING (-46.70097108746287 -23.69615686057...,c_2093,23/10/19,18:00,NaN,NaN,NaN,740.0,NaN,18,00,23,10,2019


In [21]:
print(f'{datetime.now().strftime("%H:%M:%S")}\tIniciando concatenacao dos campos de data (3 min)')

df_arestas['data_hora'] = df_arestas.apply(lambda a : 
        f"{a['dia_valor']}/{a['mes_valor']}/{a['ano_valor']} {a['hora_valor']}:{a['minuto_valor']}:00", axis = 1)

print(f'{datetime.now().strftime("%H:%M:%S")}\tFim da concatenação')

22:52:16	Iniciando concatenacao dos campos de data (3 min)
22:54:00	Fim da concatenação


In [22]:
df_arestas = df_arestas[['vertice_a', 'vertice_b', 'geometry', 'index', 
            'data_hora', 'tipo_acidente', 'qtde_vitimas',
            'tamanho_congestionamento']]

In [23]:
n = len(df_arestas) # 5362932
df_arestas = df_arestas.drop_duplicates(['vertice_a', 'vertice_b', 'data_hora', 'tipo_acidente'])

print(n - len(df_arestas), 'registros deletados') # 5062248


342325 registros deletados


In [25]:
len(df_arestas) # 5007436

5007436

In [24]:
df_arestas.to_csv('data/arestas/0_arestas_completo.csv')